In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=0d36db0f1862a059f6e10ca5cd8cc462bbd9c681166e3263dfca71743188edfd
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

file_path = '/content/drive/MyDrive/6000_axiomatic_dataset.xlsx'
os.path.exists(file_path)

True

In [ ]:
import pandas as pd

df = pd.read_excel(file_path)

In [ ]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)

In [ ]:
label_list = sorted(set(label for ex in dataset for label in ex["D_labels"].split(",")))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

In [ ]:
label2id

{'B-D': 0, 'I-D': 1, 'O': 2}

In [ ]:
id2label

{0: 'B-D', 1: 'I-D', 2: 'O'}

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer

In [ ]:
dataset = dataset.shuffle(seed=42)
dataset_split = dataset.train_test_split(test_size=0.2)
test_valid_split = dataset_split['test'].train_test_split(test_size=0.5)

# Final splits
dataset = DatasetDict({
    'train': dataset_split['train'],
    'validation': test_valid_split['train'],
    'test': test_valid_split['test'],
})

In [ ]:
def tokenize_and_align_labels(example):
    tokenized = tokenizer(example["words"].split("<w>"), is_split_into_words=True, truncation=True, padding="max_length", max_length=128)

    labels = []
    d_labels = example['D_labels'].split(",")
    word_ids = tokenized.word_ids()
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(label2id[d_labels[word_idx]])
        else:
            # For subword tokens, either repeat the label or assign -100
            labels.append(-100)
        previous_word_idx = word_idx

    tokenized["labels"] = labels
    return tokenized

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=False)

Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sent_id', 'sent', 'sent_tag', 'sent_class', 'words', 'D_labels', 'A_labels', 'R_labels', 'P_labels', 'AX_labels', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4800
    })
    validation: Dataset({
        features: ['sent_id', 'sent', 'sent_tag', 'sent_class', 'words', 'D_labels', 'A_labels', 'R_labels', 'P_labels', 'AX_labels', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 600
    })
    test: Dataset({
        features: ['sent_id', 'sent', 'sent_tag', 'sent_class', 'words', 'D_labels', 'A_labels', 'R_labels', 'P_labels', 'AX_labels', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 600
    })
})

In [ ]:
index = 2
print(tokenized_dataset['train'][index]['sent_id'])
for tok, label_id in zip(tokenizer.convert_ids_to_tokens(tokenized_dataset['train']["input_ids"][index]), tokenized_dataset['train']['labels'][index]):
    label_str = id2label[label_id] if label_id != -100 else "-100"
    print(f"{tok}\t{label_str}")

1910439
[CLS]	-100
figure	O
2	O
##b	-100
shows	O
a	O
s	O
##che	-100
##matic	-100
,	O
perspective	O
view	O
of	O
a	O
space	O
##r	-100
for	O
the	O
tube	O
spa	O
##cing	-100
and	O
fast	O
##ening	-100
system	O
of	O
figure	O
2	O
##a	-100
,	O
according	O
to	O
an	O
em	O
##bo	-100
##di	-100
##ment	-100
of	O
the	O
present	O
disclosure	O
.	O
[SEP]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-100
[PAD]	-1

In [ ]:
from seqeval.metrics import classification_report, f1_score, accuracy_score
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
args = TrainingArguments(
    output_dir="/content/BertForNER-D_Labels",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/tmp/ipython-input-18-3662767384.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: michelepapucci (michelepapucci-cnr) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.055925,0.980706,0.620805
2,0.086200,0.056195,0.981255,0.667774
3,0.086200,0.064621,0.981072,0.664360
4,0.029600,0.084404,0.981377,0.682432
5,0.010900,0.091147,0.979424,0.657188
6,0.010900,0.099583,0.981133,0.680412
7,0.004100,0.107741,0.981011,0.660839
8,0.004100,0.115613,0.981866,0.663158
9,0.002300,0.118590,0.981866,0.672504
10,0.001200,0.119154,0.981683,0.682927


TrainOutput(global_step=3000, training_loss=0.02237690273920695, metrics={'train_runtime': 1254.7536, 'train_samples_per_second': 38.255, 'train_steps_per_second': 2.391, 'total_flos': 3135589429248000.0, 'train_loss': 0.02237690273920695, 'epoch': 10.0})

In [ ]:
results = trainer.evaluate(tokenized_dataset["test"])
print("Test set results:", results)

Test set results: {'eval_loss': 0.16559042036533356, 'eval_accuracy': 0.9744984061503844, 'eval_f1': 0.5753424657534247, 'eval_runtime': 4.2559, 'eval_samples_per_second': 140.982, 'eval_steps_per_second': 8.929, 'epoch': 10.0}


In [ ]:
trainer.model.save_pretrained("/content/saved_model")

In [ ]:
model = BertForTokenClassification.from_pretrained("/content/saved_model", output_hidden_states=True)

In [ ]:
from tqdm import tqdm
import torch

In [ ]:
embeddings = []
ids = []
sentences = []
labels = []

# Extracting the representations from each sentence of the probing dataset, with model(*input_ids*), where *input_ids* corresponds to the tokenized sentence
for index, line in tqdm(df.iterrows(), total=1000):
  sent_id = index
  sentence = line['sent']
  labels.append(line['sent_tag'])

  # Tokenization of the sentence
  input_ids = tokenizer.encode(sentence, add_special_tokens=True)
  input_ids = torch.tensor([input_ids])

  # Extraction of the representations
  with torch.no_grad():
    outputs = model(input_ids)
    embeddings.append(outputs["hidden_states"])
    ids.append(sent_id)
  if index == 999:
    break


100%|█████████▉| 999/1000 [02:43<00:00,  6.11it/s]


In [ ]:
labels

['FUN',
 'FUN',
 'STR',
 'OTH',
 'STR',
 'MIX',
 'STR',
 'FUN',
 'STR',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'MIX',
 'STR',
 'FUN',
 'MIX',
 'FUN',
 'MIX',
 'STR',
 'MIX',
 'MIX',
 'MIX',
 'FUN',
 'FUN',
 'MIX',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'FUN',
 'OTH',
 'FUN',
 'FUN',
 'STR',
 'FUN',
 'FUN',
 'MIX',
 'MIX',
 'MIX',
 'FUN',
 'MIX',
 'MIX',
 'MIX',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'MIX',
 'FUN',
 'FUN',
 'MIX',
 'MIX',
 'MIX',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'STR',
 'FUN',
 'MIX',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'FUN',
 'FUN',
 'STR',
 'MIX',
 'FUN',
 'FUN',
 'OTH',
 'FUN',
 'STR',
 'FUN',
 'FUN',
 'MIX',
 'FUN',
 'STR',
 'MIX',
 'MIX',
 'MIX',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'STR',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'MIX',
 'STR',
 'FUN',
 'FUN',
 'MIX',
 'FUN',
 'MIX',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'FUN',
 'MIX',
 'FUN',
 'FUN',


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np

cls_representations = []

for layer_outputs in embeddings:
    cls_repr = [layer_output[0, 0, :].numpy() for layer_output in layer_outputs]  # one vector per layer
    cls_representations.append(cls_repr)

cls_representations = np.stack(cls_representations)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

X_train_idx, X_test_idx, y_train, y_test = train_test_split(
    np.arange(len(y)), y, test_size=0.2, random_state=42, stratify=y
)

results = {}
for layer in range(cls_representations.shape[1]):
    X = cls_representations[:, layer, :]  # Shape: (num_samples, hidden_dim)

    X_train = X[X_train_idx]
    X_test = X[X_test_idx]

    clf = LinearSVC()
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    acc = f1_score(y_test, preds, average="weighted")

    results[f"Layer {layer}"] = acc
    print(f"Layer {layer}: F1-Score = {acc:.4f}")


Layer 0: F1-Score = 0.3558
Layer 1: F1-Score = 0.5647
Layer 2: F1-Score = 0.5366
Layer 3: F1-Score = 0.5800
Layer 4: F1-Score = 0.5634
Layer 5: F1-Score = 0.5445
Layer 6: F1-Score = 0.5487
Layer 7: F1-Score = 0.5736
Layer 8: F1-Score = 0.6082
Layer 9: F1-Score = 0.5785
Layer 10: F1-Score = 0.6442
Layer 11: F1-Score = 0.6318
Layer 12: F1-Score = 0.6185
